In [490]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [491]:
dataset = pd.read_csv("dados/historicoMaterias.csv")

In [492]:
dataset

,data,url_noticia,url_noticia_curto,titulo,conteudo_noticia,assunto
0,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/m...,"Mesmo com alta do dólar, gastos de brasileiros...","\n\tA alta de 15% no dólar em 2013, a maior do...",economia
1,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/p...,"Para Dilma, é 'apressada' a tese de que emerge...",\n\tA presidente Dilma Rousseff afirmou nesta ...,economia
2,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/t...,"Temos sido capazes de reduzir a inflação', diz...","\n\tO presidente do Banco Central, Alexandre T...",economia
3,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/g...,Governo argentino autoriza compra de dólares a...,\n\tO governo argentino anunciou nesta sexta-f...,economia
4,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/p...,Petrobras desiste de parte da área de Bem-te-v...,\n\tA Petrobras afirmou que propôs à Agência N...,economia
...,...,...,...,...,...,...
10104,2018-12-22,https://web.archive.org/web/20181222014135/htt...,https://g1.globo.com/economia/tecnologia/notic...,Brasil ganha 10 milhões de internautas em 1 an...,"Em apenas 1 ano, o número de internautas no B...",tecnologia
10105,2018-12-27,https://web.archive.org/web/20181227111959/htt...,https://g1.globo.com/economia/tecnologia/notic...,Como a Coreia do Norte quer usar alta tecnolog...,A Coreia do Norte costuma ostentar seu aparat...,tecnologia
10106,2018-12-27,https://web.archive.org/web/20181227111959/htt...,https://g1.globo.com/economia/tecnologia/notic...,Venda de computadores desacelera por eleições ...,As vendas de computadores no Brasil desaceler...,tecnologia
10107,2018-12-28,https://web.archive.org/web/20181228065113/htt...,https://g1.globo.com/economia/tecnologia/notic...,"Instagram passa a mostrar posts na horizontal,...",O Instagram liberou na tarde desta quinta-fei...,tecnologia


Dropando colunas desncessárias para o objetivo da rede neural

In [493]:
dataset = dataset.drop(["url_noticia", "url_noticia_curto", "titulo", "data"], axis=1)

In [494]:
dataset

,conteudo_noticia,assunto
0,"\n\tA alta de 15% no dólar em 2013, a maior do...",economia
1,\n\tA presidente Dilma Rousseff afirmou nesta ...,economia
2,"\n\tO presidente do Banco Central, Alexandre T...",economia
3,\n\tO governo argentino anunciou nesta sexta-f...,economia
4,\n\tA Petrobras afirmou que propôs à Agência N...,economia
...,...,...
10104,"Em apenas 1 ano, o número de internautas no B...",tecnologia
10105,A Coreia do Norte costuma ostentar seu aparat...,tecnologia
10106,As vendas de computadores no Brasil desaceler...,tecnologia
10107,O Instagram liberou na tarde desta quinta-fei...,tecnologia


Como é possível observar na tabela acima, o banco de dados dos textos, ainda possuem alguns caracteres restritos, como /n, /t, /xa0 e etc. Neste caso, estarei tratando para que estes sejam removidos todos do banco de dados

In [495]:
x = 0

for string in dataset["conteudo_noticia"]:
    string = str(string).replace("\n", " ")
    string = str(string).replace("\t", " ")
    string = str(string).replace("\xa0", " ")
    dataset.iloc[x]["conteudo_noticia"] = string
    x+=1


Encurtamento prático

In [496]:
dataset

,conteudo_noticia,assunto
0,"A alta de 15% no dólar em 2013, a maior dos ...",economia
1,A presidente Dilma Rousseff afirmou nesta se...,economia
2,"O presidente do Banco Central, Alexandre Tom...",economia
3,O governo argentino anunciou nesta sexta-fei...,economia
4,A Petrobras afirmou que propôs à Agência Nac...,economia
...,...,...
10104,"Em apenas 1 ano, o número de internautas no B...",tecnologia
10105,A Coreia do Norte costuma ostentar seu aparat...,tecnologia
10106,As vendas de computadores no Brasil desaceler...,tecnologia
10107,O Instagram liberou na tarde desta quinta-fei...,tecnologia


Para que a tokenization seja efetiva, preciso tokenizar o texto todo existente no meu dataset. Este banco de dados possuí blocos de texto, ao invés de ser um texto corrido completo, neste caso, é por esta razão que eu somo o texto todo para então passar o tokenizador. Se eu passar ele individualmente em cada bloco de texto, cada linha do meu dataset teria uma tokenização diferente, ou seja, a mesma palavra em diferentes blocos de texto teria 2 tokens

In [497]:
corpus = ''

for string in dataset["conteudo_noticia"]:
    corpus += string

In [498]:
corpus = corpus.lower().split("\n")

In [499]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

Quantidade total de palavras detectadas no texto

In [500]:
len(tokenizer.word_index)

92754

Nesta parte, eu pego o dataset original, e vou converte-lo para tokens

In [501]:
for x in range(len(dataset["conteudo_noticia"])):
    dataset["conteudo_noticia"].iloc[x] = tokenizer.texts_to_sequences([dataset["conteudo_noticia"].iloc[x]])[0]

In [502]:
assunto = []

for i in range(len(dataset["assunto"])): 
    for x in range(dataset["assunto"].nunique()):
        if dataset["assunto"].iloc[i] == dataset["assunto"].unique()[x]:
            assunto.append(x)

assunto = np.array(assunto)
dataset["assunto"] = assunto


In [503]:
dataset

,conteudo_noticia,assunto
0,"[3, 314, 1, 250, 10, 663, 8, 632, 3, 117, 24, ...",0
1,"[3, 49, 659, 1367, 90, 58, 185, 37, 542, 157, ...",0
2,"[2, 49, 6, 216, 410, 1066, 13918, 56, 58, 185,...",0
3,"[2, 74, 981, 563, 58, 185, 37, 542, 5, 6706, 3...",0
4,"[3, 608, 90, 5, 4515, 34, 872, 166, 6, 938, 22...",0
...,...,...
10104,"[8, 107, 54, 43, 2, 287, 1, 5339, 10, 53, 1608...",4
10105,"[3, 2953, 6, 510, 2423, 24378, 62, 18664, 1411...",4
10106,"[23, 1027, 1, 3863, 10, 53, 26470, 10, 387, 56...",4
10107,"[2, 1834, 6263, 12, 514, 178, 187, 37, 419, 17...",4


In [504]:
maxContent = max(len(tokens) for tokens in dataset["conteudo_noticia"])

In [505]:
for conteudo in dataset["conteudo_noticia"]:
    if len(conteudo) > 0:
        for x in range(maxContent - len(conteudo)): 
            conteudo.append(0)

In [506]:
dataset

,conteudo_noticia,assunto
0,"[3, 314, 1, 250, 10, 663, 8, 632, 3, 117, 24, ...",0
1,"[3, 49, 659, 1367, 90, 58, 185, 37, 542, 157, ...",0
2,"[2, 49, 6, 216, 410, 1066, 13918, 56, 58, 185,...",0
3,"[2, 74, 981, 563, 58, 185, 37, 542, 5, 6706, 3...",0
4,"[3, 608, 90, 5, 4515, 34, 872, 166, 6, 938, 22...",0
...,...,...
10104,"[8, 107, 54, 43, 2, 287, 1, 5339, 10, 53, 1608...",4
10105,"[3, 2953, 6, 510, 2423, 24378, 62, 18664, 1411...",4
10106,"[23, 1027, 1, 3863, 10, 53, 26470, 10, 387, 56...",4
10107,"[2, 1834, 6263, 12, 514, 178, 187, 37, 419, 17...",4


Escrevo o novo dataset após tokenizado, por questões de segurança

In [510]:
import csv 
with open('dados/historicoMateriasTratado.csv', 'w', newline='') as csvfile:

    header = ["assunto", "conteudo_noticia"]
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(header)

    for x in range (len(dataset)):       
        # Verifico se existem células vazias, e se sim elas não são inseridas na versão tratada dos dados, pois
        # se fossem poderiam poluir a compreensão da rede neural
        if(len(dataset["conteudo_noticia"].iloc[x]) != 0):           
            writer.writerow(dataset.iloc[x])

In [511]:
import csv 
with open('dados/assuntoTratado.csv', 'w', newline='') as csvfile:

    writer = csv.writer(csvfile, quoting=csv.QUOTE_NONE)

    for x in range (len(dataset)):       
        # Verifico se existem células vazias, e se sim elas não são inseridas na versão tratada dos dados, pois
        # se fossem poderiam poluir a compreensão da rede neural
        if(len(dataset["conteudo_noticia"].iloc[x]) != 0):           
            writer.writerow([dataset["assunto"].iloc[x]])

In [512]:
import csv 
with open('dados/conteudoTratado.csv', 'w', newline='') as csvfile:

    writer = csv.writer(csvfile, delimiter=',')

    for x in range (len(dataset)):       
        # Verifico se existem células vazias, e se sim elas não são inseridas na versão tratada dos dados, pois
        # se fossem poderiam poluir a compreensão da rede neural
        if(len(dataset["conteudo_noticia"].iloc[x]) != 0):           
            writer.writerow(dataset["conteudo_noticia"].iloc[x])

In [513]:
import json

with open("tokenizer.json", "w") as jsonfile:
    json.dump(tokenizer.to_json(), jsonfile)

In [514]:
dataset

,conteudo_noticia,assunto
0,"[3, 314, 1, 250, 10, 663, 8, 632, 3, 117, 24, ...",0
1,"[3, 49, 659, 1367, 90, 58, 185, 37, 542, 157, ...",0
2,"[2, 49, 6, 216, 410, 1066, 13918, 56, 58, 185,...",0
3,"[2, 74, 981, 563, 58, 185, 37, 542, 5, 6706, 3...",0
4,"[3, 608, 90, 5, 4515, 34, 872, 166, 6, 938, 22...",0
...,...,...
10104,"[8, 107, 54, 43, 2, 287, 1, 5339, 10, 53, 1608...",4
10105,"[3, 2953, 6, 510, 2423, 24378, 62, 18664, 1411...",4
10106,"[23, 1027, 1, 3863, 10, 53, 26470, 10, 387, 56...",4
10107,"[2, 1834, 6263, 12, 514, 178, 187, 37, 419, 17...",4
